In [7]:
!pip install networkx matplotlib pyvis openai python-dotenv



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from typing import List, Optional
import json
from openai import OpenAI
import json
import networkx as nx
import matplotlib.pyplot as plt

from dotenv import load_dotenv
import os

load_dotenv()

# Define your API key securely
# @ANISHA - Enter your OpenAI API key here
# - If you don't have an API key, you can get one by signing up at https://platform.openai.com/signup
API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = OpenAI(api_key=API_KEY)



def extract_information(text: str):
    completion = client.chat.completions.create(
        model="gpt-4-turbo",  # Use an appropriate model
        messages=[
            {"role": "system", "content": "You are an assistant specialized in extracting subjects, verbs, and objects from text for knowledge graph construction. Your task is to identify these entities, their attributes, and the relationships between them, and output them in a structured JSON format. IMPORTANT: Make sure that all pronouns are converted to nouns unless not possible. Any conjunctions should be treated as separate sentences. If a subject or object cannot be identified, label it as unidentified. Consider a verb as the connection between the subject and object, including prepositions (or both together) when relevant REMOVE over-wordiness where possible, like it doesn't have to be 'a boy named jack' as subject, it can be 'jack' as subject. If a subject or object cannot be identified, label it as unidentified. Keep everything lower case and present tense and try to keep the subject and object as a single word if possible."},
            {"role": "user", "content": f"Analyze the following text and identify subjects, verbs, and objects. For each sentence in the text, create one JSON object with the keys 'subject', 'verb', and 'object'. Only keep adjectives if they tend to act as subject or object. For example Jack is brave shall include brave. But Brave Jack is working at a farm shall not include brave. Convert pronouns to nouns where possible. If a subject or object cannot be identified, label it as unidentified.\n{text}"}
        ]
    )


    if completion.choices:
        response_message = str(completion.choices[0].message.content)
        # process response_message from string to JSON
        response_message = json.loads(response_message)
        return response_message
    else:
        print("No response received.")
        return None




In [10]:
from pyvis.network import Network
import json

def visualize_json(json_data):
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

    if isinstance(json_data, str):
        data = json.loads(json_data)
    else:
        data = json_data

    for entry in data:
        subject = entry.get('subject')
        verb = entry.get('verb')
        object = entry.get('object')

        net.add_node(subject, title=subject, color='skyblue')
        net.add_node(object, title=object, color='lightgreen')
        net.add_edge(subject, object, title=verb)

    net.show("knowledge_graph.html", notebook=False)



# Example text to analyze
text = "According to my knowledge, Mihai Surdeanu is an Associate Professor in the departments of Cognitive Science - GIDP, Computer Science, and BIO5 Institute at the University of Arizona. He earned his Ph.D. in Computer Science from Southern Methodist University in 2001 and has over 15 years of experience in building systems driven by natural language processing (NLP) and machine learning. Surdeanu has published over 80 peer-reviewed articles and has been a leader or member of teams that ranked in the top three at seven highly competitive international evaluations of end-user NLP systems such as question answering and information extraction. His work has been funded by several government organizations and private foundations. Surdeanu's research focuses on NLP and machine learning."
# Extract information from the text
json_data = extract_information(text)
print(json_data)

# Visualize the extracted information as a knowledge graph
visualize_json(json_data)


[{'subject': 'mihai surdeanu', 'verb': 'is', 'object': 'associate professor'}, {'subject': 'mihai surdeanu', 'verb': 'is in', 'object': 'departments'}, {'subject': 'departments', 'verb': 'include', 'object': 'cognitive science - gidp'}, {'subject': 'departments', 'verb': 'include', 'object': 'computer science'}, {'subject': 'departments', 'verb': 'include', 'object': 'bio5 institute'}, {'subject': 'mihai surdeanu', 'verb': 'earned', 'object': 'ph.d.'}, {'subject': 'ph.d.', 'verb': 'is in', 'object': 'computer science'}, {'subject': 'ph.d.', 'verb': 'is from', 'object': 'southern methodist university'}, {'subject': 'ph.d.', 'verb': 'is from', 'object': '2001'}, {'subject': 'mihai surdeanu', 'verb': 'has', 'object': 'experience'}, {'subject': 'experience', 'verb': 'is over', 'object': '15 years'}, {'subject': 'systems', 'verb': 'are driven by', 'object': 'nlp'}, {'subject': 'systems', 'verb': 'are driven by', 'object': 'machine learning'}, {'subject': 'mihai surdeanu', 'verb': 'has publi

In [11]:
from pyvis.network import Network
import json

def visualize_comparison(source_json_data, output_json_data, display='both'):
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

    # Parse JSON data
    source_data = json.loads(source_json_data) if isinstance(source_json_data, str) else source_json_data
    output_data = json.loads(output_json_data) if isinstance(output_json_data, str) else output_json_data

    # Extract nodes and verbs from data
    source_nodes = set(entry['subject'] for entry in source_data) | set(entry['object'] for entry in source_data)
    output_nodes = set(entry['subject'] for entry in output_data) | set(entry['object'] for entry in output_data)
    
    # Determine common nodes
    common_nodes = source_nodes & output_nodes
    
    # Function to add nodes and edges to the network
    def add_nodes_edges(data, node_color):
        for entry in data:
            subject, verb, object = entry['subject'], entry['verb'], entry['object']
            # Conditionally color nodes if they are common
            sub_color = 'yellow' if subject in common_nodes else node_color
            obj_color = 'yellow' if object in common_nodes else node_color
            net.add_node(subject, title=subject, color=sub_color)
            net.add_node(object, title=object, color=obj_color)
            net.add_edge(subject, object, title=verb)

    # Add source graph nodes and edges
    if display in ['source', 'both']:
        add_nodes_edges(source_data, 'blue')

    # Add output graph nodes and edges
    if display in ['output', 'both']:
        add_nodes_edges(output_data, 'green')

    # Save and show the graph
    net.show("knowledge_graph_comparison.html", notebook=False)

# Example JSON data for source and output
source_json = json.dumps([
    {"subject": "Fox", "verb": "jumps", "object": "Dog"},
    {"subject": "Dog", "verb": "barks", "object": "loudly"}
])

output_json = json.dumps([
    {"subject": "Fox", "verb": "runs", "object": "fast"},
    {"subject": "Dog", "verb": "barks", "object": "quietly"}
])

# Visualize comparison allowing the user to choose which parts to display
visualize_comparison(source_json, output_json, display='both')


knowledge_graph_comparison.html


In [12]:
# Generate source text
source_text = "There was once a boy named Jack who lived in a small village. Jack was known for his bravery and kindness. One day, Jack decided to climb a giant beanstalk that had grown in his backyard. At the top of the beanstalk, he found a castle inhabited by a giant. The giant had a magical hen that laid golden eggs. Jack managed to steal the hen and escape from the giant. The giant chased Jack down the beanstalk, but Jack managed to cut it down, causing the giant to fall to his doom. Jack returned to his village with the magical hen and lived happily ever after."

# Generate output text with subtle hallucinations
output_text = "In the town of Kingman, Arizona, there lived a courageous and kind-hearted boy named Jack. One adventurous day, he ascended a massive beanstalk sprouting in his backyard. At its peak, Jack discovered a castle, home to a formidable giant. This giant possessed a hen with the ability to produce golden eggs. Jack cleverly captured the hen and fled from the giant's clutches. In a thrilling chase, the giant pursued Jack along the beanstalk. In a daring move, Jack chopped down the beanstalk, leading to the giant's tragic fall and demise. Triumphantly, Jack returned to his village, where he and the magical hen enjoyed a prosperous and joyful life."

# Extract information from source text
source_json = extract_information(source_text)
output_json = extract_information(output_text)

print("Source JSON:", source_json)
print("Output JSON:", output_json)

# Compare the graphs
visualize_comparison(source_json, output_json, display='both')


Source JSON: [{'subject': 'jack', 'verb': 'live', 'object': 'village'}, {'subject': 'jack', 'verb': 'be known', 'object': 'bravery kindness'}, {'subject': 'jack', 'verb': 'decide', 'object': 'climb'}, {'subject': 'beanstalk', 'verb': 'grow', 'object': 'backyard'}, {'subject': 'jack', 'verb': 'find', 'object': 'castle'}, {'subject': 'castle', 'verb': 'be inhabited', 'object': 'giant'}, {'subject': 'giant', 'verb': 'have', 'object': 'hen'}, {'subject': 'hen', 'verb': 'lay', 'object': 'eggs'}, {'subject': 'jack', 'verb': 'manage to steal', 'object': 'hen'}, {'subject': 'jack', 'verb': 'escape', 'object': 'giant'}, {'subject': 'giant', 'verb': 'chase', 'object': 'jack'}, {'subject': 'jack', 'verb': 'manage to cut down', 'object': 'beanstalk'}, {'subject': 'giant', 'verb': 'fall', 'object': 'doom'}, {'subject': 'jack', 'verb': 'return', 'object': 'village'}, {'subject': 'jack', 'verb': 'live', 'object': 'happily'}]
Output JSON: [{'subject': 'jack', 'verb': 'live', 'object': 'kingman'}, {'su